Messy data often has structual issues, known as untidy data, each observation is a row and each variable is a column, each observational unit forms a table
Dirty data often has content issues, known as low quality data, for example, duplicate data, inaccuracies, corruptions

__cleaning process__

df.info()

bullet point issues by quality and messiness. 
quality - spelling, duplicates, inaccuracies

In [64]:
import pandas as pd

In [65]:
patients = pd.read_csv('./auralin_dataset/patients.csv')
treatments = pd.read_csv('./auralin_dataset/treatments.csv')
adverse_reactions = pd.read_csv('./auralin_dataset/adverse_reactions.csv')

# Assess

In [66]:
# Display the patients table
patients.head()

,patient_id,assigned_sex,given_name,surname,address,city,state,zip_code,country,contact,birthdate,weight,height,bmi
0,1,female,Zoe,Wellish,576 Brown Bear Drive,Rancho California,California,92390.0,United States,951-719-9170ZoeWellish@superrito.com,7/10/1976,121.7,66,19.6
1,2,female,Pamela,Hill,2370 University Hill Road,Armstrong,Illinois,61812.0,United States,PamelaSHill@cuvox.de+1 (217) 569-3204,4/3/1967,118.8,66,19.2
2,3,male,Jae,Debord,1493 Poling Farm Road,York,Nebraska,68467.0,United States,402-363-6804JaeMDebord@gustr.com,2/19/1980,177.8,71,24.8
3,4,male,Liêm,Phan,2335 Webster Street,Woodbridge,NJ,7095.0,United States,PhanBaLiem@jourrapide.com+1 (732) 636-8246,7/26/1951,220.9,70,31.7
4,5,male,Tim,Neudorf,1428 Turkey Pen Lane,Dothan,AL,36303.0,United States,334-515-7487TimNeudorf@cuvox.de,2/18/1928,192.3,27,26.1


In [67]:
treatments.head()

,given_name,surname,auralin,novodra,hba1c_start,hba1c_end,hba1c_change
0,veronika,jindrová,41u - 48u,-,7.63,7.20,NaN
1,elliot,richardson,-,40u - 45u,7.56,7.09,0.97
2,yukitaka,takenaka,-,39u - 36u,7.68,7.25,NaN
3,skye,gormanston,33u - 36u,-,7.97,7.62,0.35
4,alissa,montez,-,33u - 29u,7.78,7.46,0.32


In [68]:
adverse_reactions.head()

,given_name,surname,adverse_reaction
0,berta,napolitani,injection site discomfort
1,lena,baer,hypoglycemia
2,joseph,day,hypoglycemia
3,flavia,fiorentino,cough
4,manouck,wubbels,throat irritation


# Checking Quality

first we can cheeck the treatments table, firstly let's check the change in hba1c (hba1c is formed when haemoglobin and glucuse bind, hence more sugar in the bloodstream means more HBA1C)

In [69]:
treatments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 280 entries, 0 to 279
Data columns (total 7 columns):
given_name      280 non-null object
surname         280 non-null object
auralin         280 non-null object
novodra         280 non-null object
hba1c_start     280 non-null float64
hba1c_end       280 non-null float64
hba1c_change    171 non-null float64
dtypes: float64(3), object(4)
memory usage: 15.4+ KB


#### Quality
- `treatment` table - missing hba1c missing values
- `patients` - zip_code - shouldn't be decimal, should be a string
- `patients` - some entries have 4 chars, should be 5 chars, so leading zeros aren't being added
- `patients` - some height entries are too short (27 inches - Tim Neudorf), probably 72 inches
- `patients` - New York and NY represent the same place, some full name some using abbreviations
- `treatment` - u format, start and end should be in separate columns
- `treatment` - lower case names, but same names have upper first character in the  patients
- `adverse_reactions` - lower case names, but same names have upper first character in the 
- `treatment` - missing data, should be 250 rows
- `patients` - nan for some addresses and contact
- `patients` - datetime object
- `treatments` - hba1c_change for row 1 (starting from 0) for elliot, has large change of 0.97
- `patients` - multiple phone number formats in the contacts columns

In [70]:
patients.query('height==27')

,patient_id,assigned_sex,given_name,surname,address,city,state,zip_code,country,contact,birthdate,weight,height,bmi
4,5,male,Tim,Neudorf,1428 Turkey Pen Lane,Dothan,AL,36303.0,United States,334-515-7487TimNeudorf@cuvox.de,2/18/1928,192.3,27,26.1


feature engineering - creating new descriptive features from your dataset

## Checking date format in patients

Some entries are recorded as m/d/y instead of d/m/y

In [71]:
from datetime import datetime as dt
def convert_date(date_str):
    return dt.strptime(date_str, "%m/%d/%Y")
patients['birthdate'].apply(convert_date)

0     1976-07-10
1     1967-04-03
2     1980-02-19
3     1951-07-26
4     1928-02-18
5     1931-08-31
6     1969-11-19
7     1958-08-13
8     1937-03-06
9     1930-12-03
10    1974-07-16
11    1954-02-03
12    1957-08-05
13    1961-03-07
14    1997-08-15
15    1922-08-23
16    1952-11-14
17    1922-07-24
18    1949-09-04
19    1988-12-13
20    1934-09-24
21    1983-04-12
22    1936-01-19
23    1960-05-26
24    1985-08-01
25    1922-06-19
26    1933-12-19
27    1959-04-10
28    1937-06-26
29    1985-08-01
         ...    
473   1998-07-18
474   1961-10-07
475   1953-09-25
476   1958-12-02
477   1966-09-29
478   1977-06-24
479   1955-09-15
480   1938-03-25
481   1930-08-10
482   1945-03-03
483   1987-08-14
484   1934-05-31
485   1973-02-15
486   1983-07-06
487   1962-11-07
488   1970-01-27
489   1988-12-01
490   1953-05-29
491   1950-09-21
492   1970-05-23
493   1997-03-18
494   1942-01-16
495   1972-09-05
496   1942-09-12
497   1937-08-19
498   1959-04-10
499   1948-03-26
500   1971-01-

In [72]:
patients[patients['patient_id']==9]['given_name']

8    Dsvid
Name: given_name, dtype: object

In [73]:
treatments[treatments.duplicated()]

,given_name,surname,auralin,novodra,hba1c_start,hba1c_end,hba1c_change
136,joseph,day,29u - 36u,-,7.7,7.19,NaN


In [74]:
patients[patients.address.duplicated()]

,patient_id,assigned_sex,given_name,surname,address,city,state,zip_code,country,contact,birthdate,weight,height,bmi
29,30,male,Jake,Jakobsen,648 Old Dear Lane,Port Jervis,New York,12771.0,United States,JakobCJakobsen@einrot.com+1 (845) 858-7707,8/1/1985,155.8,67,24.4
219,220,male,Mỹ,Quynh,NaN,NaN,NaN,NaN,NaN,NaN,4/9/1978,237.8,69,35.1
229,230,male,John,Doe,123 Main Street,New York,NY,12345.0,United States,johndoe@email.com1234567890,1/1/1975,180.0,72,24.4
230,231,female,Elisabeth,Knudsen,NaN,NaN,NaN,NaN,NaN,NaN,9/23/1976,165.9,63,29.4
234,235,female,Martina,Tománková,NaN,NaN,NaN,NaN,NaN,NaN,4/7/1936,199.5,65,33.2
237,238,male,John,Doe,123 Main Street,New York,NY,12345.0,United States,johndoe@email.com1234567890,1/1/1975,180.0,72,24.4
242,243,male,John,O'Brian,NaN,NaN,NaN,NaN,NaN,NaN,2/25/1957,205.3,74,26.4
244,245,male,John,Doe,123 Main Street,New York,NY,12345.0,United States,johndoe@email.com1234567890,1/1/1975,180.0,72,24.4
249,250,male,Benjamin,Mehler,NaN,NaN,NaN,NaN,NaN,NaN,10/30/1951,146.5,69,21.6
251,252,male,John,Doe,123 Main Street,New York,NY,12345.0,United States,johndoe@email.com1234567890,1/1/1975,180.0,72,24.4


## Tidiness

- contact column in patients includes both email and phoen number, needs to be split
- treatments table, each row has an entry for both auralin and novodra, but each obseration will be for either auralin or novodra and not both. Better to have this information in 3 separate columns, the treatment, the start dose and the end dose

In [94]:
def get_dosage(dosage_str):
    return [int(dose.replace('u','')) for dose in dosage_str.split('-')]
def test(row):
    dose_string = ''
    if(row['novodra']=='-'):
        # it's auralin
        row['treatment'] = dose_string = 'auralin'
    else:
        row['treatment'] = dose_string = 'novodra'
        
    start_stop_dosage = get_dosage(row[dose_string])
    row['start_dose'] = start_stop_dosage[0]
    row['end_dose'] = start_stop_dosage[1]
    return row


treatments = treatments.apply(test, axis=1)
treatments.drop(axis=1, labels=['auralin', 'novodra'], inplace=True)


In [95]:
treatments

,given_name,surname,hba1c_start,hba1c_end,hba1c_change,treatment,start_dose,end_dose
0,veronika,jindrová,7.63,7.20,NaN,auralin,41,48
1,elliot,richardson,7.56,7.09,0.97,novodra,40,45
2,yukitaka,takenaka,7.68,7.25,NaN,novodra,39,36
3,skye,gormanston,7.97,7.62,0.35,auralin,33,36
4,alissa,montez,7.78,7.46,0.32,novodra,33,29
5,jasmine,sykes,7.56,7.18,0.38,novodra,42,44
6,sophia,haugen,7.65,7.27,0.38,auralin,37,42
7,eddie,archer,7.89,7.55,0.34,auralin,31,38
8,saber,ménard,8.08,7.70,NaN,novodra,54,54
9,asia,woźniak,7.76,7.37,NaN,auralin,30,36


In [96]:
patients.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 503 entries, 0 to 502
Data columns (total 14 columns):
patient_id      503 non-null int64
assigned_sex    503 non-null object
given_name      503 non-null object
surname         503 non-null object
address         491 non-null object
city            491 non-null object
state           491 non-null object
zip_code        491 non-null float64
country         491 non-null object
contact         491 non-null object
birthdate       503 non-null object
weight          503 non-null float64
height          503 non-null int64
bmi             503 non-null float64
dtypes: float64(3), int64(2), object(9)
memory usage: 55.1+ KB


In [97]:
treatments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 280 entries, 0 to 279
Data columns (total 8 columns):
given_name      280 non-null object
surname         280 non-null object
hba1c_start     280 non-null float64
hba1c_end       280 non-null float64
hba1c_change    171 non-null float64
treatment       280 non-null object
start_dose      280 non-null int64
end_dose        280 non-null int64
dtypes: float64(3), int64(2), object(3)
memory usage: 17.6+ KB


In [98]:
adverse_reactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34 entries, 0 to 33
Data columns (total 3 columns):
given_name          34 non-null object
surname             34 non-null object
adverse_reaction    34 non-null object
dtypes: object(3)
memory usage: 896.0+ bytes


In [99]:
patients.head()

,patient_id,assigned_sex,given_name,surname,address,city,state,zip_code,country,contact,birthdate,weight,height,bmi
0,1,female,Zoe,Wellish,576 Brown Bear Drive,Rancho California,California,92390.0,United States,951-719-9170ZoeWellish@superrito.com,7/10/1976,121.7,66,19.6
1,2,female,Pamela,Hill,2370 University Hill Road,Armstrong,Illinois,61812.0,United States,PamelaSHill@cuvox.de+1 (217) 569-3204,4/3/1967,118.8,66,19.2
2,3,male,Jae,Debord,1493 Poling Farm Road,York,Nebraska,68467.0,United States,402-363-6804JaeMDebord@gustr.com,2/19/1980,177.8,71,24.8
3,4,male,Liêm,Phan,2335 Webster Street,Woodbridge,NJ,7095.0,United States,PhanBaLiem@jourrapide.com+1 (732) 636-8246,7/26/1951,220.9,70,31.7
4,5,male,Tim,Neudorf,1428 Turkey Pen Lane,Dothan,AL,36303.0,United States,334-515-7487TimNeudorf@cuvox.de,2/18/1928,192.3,27,26.1


You also may need to consolidate tables together, you can see treatments needs to be joined to with the adverse reactions table